In [3]:
import pandas as pd
import requests

In [4]:
def get_osm_data(city, city_part, osm_attribute, file_naming):
    
    if city_part == '':
        #Create Overpass API Query
        overpass_url = "http://overpass-api.de/api/interpreter"
        overpass_query = f"""
            [out:json];
            area[name='{city}'];
            nwr[{osm_attribute}](area);
            out center;
        """

    else:
        #Create Overpass API Query
        overpass_url = "http://overpass-api.de/api/interpreter"
        overpass_query = f"""
            [out:json];
            area[name='{city}']->.a;
            (
                rel[name='{city_part}'](area.a);
                nwr[{osm_attribute}](area.a);
            );
            out center;
        """

    #Perform Overpass API Request
    response = requests.get(overpass_url, params={'data': overpass_query})
    response_content = response.json()
    print('Response acquired!')


    #Collect Relevant Data
    csv_data = []
    for element in response_content['elements']:
        if element['type'] == 'node':
            csv_data.append({
                'id': element['id'],
                'lat': element['lat'],
                'lon': element['lon'],
                'name': element['tags'].get('name', ''),
                'building': element['tags'].get('building', ''),
                'addr:street': element['tags'].get('addr:street', ''),
                'addr:housenumber': element['tags'].get('addr:housenumber', ''),
                'addr:postcode': element['tags'].get('addr:postcode', ''),
            })

        else:
            csv_data.append({  
                'id': element['id'],
                'lat': element['center']['lat'],
                'lon': element['center']['lon'],
                'name': element['tags'].get('name', ''),
                'building': element['tags'].get('building', ''),
                'addr:street': element['tags'].get('addr:street', ''),
                'addr:housenumber': element['tags'].get('addr:housenumber', ''),
                'addr:postcode': element['tags'].get('addr:postcode', ''),

            })


    #Transform to DataFrame and save file
    df = pd.DataFrame(csv_data)
    if city_part == '':
        df.to_csv(f'./OSM_Data/{city}-{file_naming}.csv', index=False)
        print(f'Successfully saved {city}-{file_naming}!')
    else:
        df.to_csv(f'./OSM_Data/{city_part}-{file_naming}.csv', index=False)
        print(f'Successfully saved {city_part}-{file_naming}!')

In [ ]:
city_parts = [
         'Zellerau',
         'Dürrbachtal',
         'Grombühl', 
         'Lindleinsmühle', 
         'Frauenland', 
         'Sanderau', 
         'Heidingsfeld', 
         'Heuchelhof', 
         'Steinbachtal', 
         'Versbach',
         'Lengfeld',
         'Rottenbauer'
         ]

#Zusätzliche Daten Würzburg
get_osm_data('Würzburg', '', "amenity='parcel_locker'", 'Packstationen')
get_osm_data('Würzburg', '', 'amenity="pharmacy"', 'Apotheken')

#Gebäudedaten Würzburgs
get_osm_data('Würzburg Altstadt', '', 'building', 'Buildings')
for city_part in city_parts:
    get_osm_data('Würzburg', city_part, 'building', 'Buildings')




In [6]:
get_osm_data('Würzburg', '', 'building', 'Buildings')

Response acquired!
Successfully saved Würzburg-Buildings!
